In [4]:
import numpy as np
from numpy import linalg as la
#####################################################################################
#################Implicit Finite Differences Method on S=exp(x)######################
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
#########################===========!The Function!==========#########################
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
#This function will return the FairValue of a Vanilla European Call/Put
# S_max: is a high Stock Price, that cannot be reached
# T:     is the maturity
# r:     is the risk free rate
# q:     is the dividend yield
# sigma: is the volatility of the stock
# N:     is the time discretization number
# M:     is the stock discretization parameter
# isCall:is the derivative type call or put
# isAmerican: Trivial
def VanillaOption_Implicit(
                  S_0,
                  T,
                  K,
                  r,
                  q,
                  sigma,
                  N,
                  M,
                  isCall,
                  isAmerican):
    outVector = np.zeros((2,1),dtype = float)
    S_max = 2.0*K
    S_min = 0.0
    #Discretization of the Stock and the Time
    dS=(S_max-S_min)/float(M)  
    dt=T/float(N)
    
    j=np.arange(0,M,dtype=np.float)   
    
    #'Transition Probabilities'
    a=(0.5*(r-q)*j*dt-0.5*sigma**2*j**2*dt)
    b=(1.0+sigma**2*j**2*dt+r*dt)
    c=(-0.5*(r-q)*j*dt-0.5*sigma**2*j**2*dt)
    #The matrix from the transition probabilities
    A=np.diag(b)+np.diag(a[1:],k=-1)+np.diag(c[0:M-1],k=1)
    B=la.inv(A)
    infinityNormOfB = la.norm(B,np.inf)
    outVector[1] = infinityNormOfB
    #The matrix for the option price
    #N+1: Time Steps
    #M+1: Stock Steps
    FV=np.zeros((N+1,M+1))
    
    #Set up the boundary conditions//Contract dependent :-(
    if(bool(isCall)):
        FV[:,0]=S_min
        FV[:,M]=[S_max * np.exp(-r*( N - j)*dt) for j in range(N+1)]
        FV[N,:]=np.maximum(np.arange(S_min,S_max+dS/2.0,dS,dtype=np.float)-K,0)
    else:
        FV[:,0]=[K * np.exp(-r*( N - j)*dt) for j in range(N+1)]
        FV[:,M]=0
        FV[N,:]=np.maximum(K-np.arange(S_min,S_max+dS/2.0,dS,dtype=np.float),0)
    
    FV=np.matrix(np.array(FV))
    #Start the Backward Iteration
    for i in range(N-1,-1,-1):
        k_i=np.zeros((M,1))
        #Inserts the first and the last element
        k_i[0]=-a[0]*FV[i+1,0] 
        k_i[M-1]=-c[M-1]*FV[i+1,M]
        
        FV[i,0:M]=np.dot(FV[i+1,0:M]+np.matrix(np.array(k_i)).transpose(),B)
        
        if(bool(isAmerican)):
            if(bool(isCall)):
                FV[i,:] = np.maximum(np.arange(0,S_max+dS/2,dS,dtype=float)-K,FV[i,:]) 
            else:
                FV[i,:] = np.maximum(K-np.arange(0,S_max+dS/2,dS,dtype=float),FV[i,:])
        outVector[0] = FV[0,(FV.shape[1]+1)/2]
    return outVector
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
#########################========End Of The Function========#########################
#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#--#
S_0 = 50.0
T=1.0
K=50.0
r=0.1
q=0.00
sigma = 0.25
N=100
M=40
isCall = 1
isAmerican = 0
print VanillaOption_Implicit(
                    S_0,
                    T,
                    K,
                    r,
                    q,
                    sigma,
                    N,
                    M,
                    isCall,
                    isAmerican)        


[[ 6.77155472]
 [ 0.99937502]]
